# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
df = pd.read_csv("Orders.zip")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [6]:
df["CustomerID"].nunique()

4339

In [17]:
spent_by_client = df.groupby(["CustomerID"])["amount_spent"].agg("sum").sort_values(ascending=False).reset_index()
q95 = spent_by_client["amount_spent"].quantile(.95)
q75 = spent_by_client["amount_spent"].quantile(.75)

In [18]:
def categorize (number):
    if number >= q95: return "VIP"
    elif number >= q75: return "Preferred"
    else: return


In [35]:
spent_by_client["category"] = spent_by_client["amount_spent"].apply(categorize)
spent_by_client.sample(10)

,CustomerID,amount_spent,category
1216,16348,1469.45,None
383,13183,3957.78,Preferred
2681,17880,458.92,None
2896,16776,388.28,None
647,13851,2651.46,Preferred
102,16180,10254.18,VIP
1241,16103,1429.64,None
1243,14035,1428.02,None
46,15159,18641.01,VIP
16,15769,56252.72,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [46]:
spent_by_client_country = spent_by_client.merge(df[["CustomerID", "Country"]], on="CustomerID", how="left").drop_duplicates()
vips = spent_by_client_country[spent_by_client_country["category"] == "VIP"]
vips.groupby(["Country"])["amount_spent"].agg("sum").reset_index().max()

Country         United Kingdom
amount_spent        3417798.95
dtype: object

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [50]:
spent_by_client_country = spent_by_client.merge(df[["CustomerID", "Country"]], on="CustomerID", how="left").drop_duplicates()
vip_and_preferred = spent_by_client_country[(spent_by_client_country["category"] == "VIP") | (spent_by_client_country["category"] == "Preferred")]
vip_and_preferred.groupby(["Country"])["amount_spent"].agg("sum").reset_index().max()

Country         United Kingdom
amount_spent       5624512.711
dtype: object